In [8]:
import discogs_client
from dotenv import load_dotenv
import requests
import os
import time
import json
import pandas
import urllib.request
from lastfm_get_album import get_album_info
load_dotenv() 
DISCOGS_CLIENTNAME= os.getenv("DISCOGS_CLIENTNAME")
DISCOGS_TOKEN=os.getenv("DISCOGS_TOKEN")

In [9]:
d = discogs_client.Client(user_agent=DISCOGS_CLIENTNAME
                          ,user_token=DISCOGS_TOKEN)

me=d.identity()
print(me)

<User 28001809 'cornelas90'>


In [11]:
import musicbrainzngs as mbz
mbz.set_useragent("cornstarch", "1.0", "cornstarch.me")

In the following cell, a release (item.release) will return a JSON with the following datapoints of interest:
- Discogs release id (id)
  - Release API Link (https:/api.discogs.com/releases/123456)
- Discogs master id
  - Master release API link (https://api.discogs.com/masters/123456)
- Cover image, hosted by discogs
- Title
- Year of Release
- Artist
- Artist id (and json link, https://api.discogs.com/artists/123456)
- Genres (array)
- Styles, or subgeneres (array)

The Release API link has the following datapoints of interest (notice, these are release specific. Probably best to query Metacritic)
- Community data
  - Ratings, count and average
  - Have #
  - Want # 

THe Master API Link has the following datapoints of interest: 
- Track data
- Year (original release)
- Genre
- Styles (more rich than release, likely)

Discogs is mainly a tool for tracking consumer goods which are versioned over time (releases). I don't think we will be that interested in querying the exact release we have on hand. I could be wrong!
But for now, lets stick to Master and pull Artist, Track, OG RElease, Genre, and Styles. 

In [26]:


for item in me.collection_folders[0].releases:
    release = item.release
    raw = release.data
    release_json_str = json.dumps(raw, indent=4)
    release_json = json.loads(release_json_str)

    master_url = json.dumps(release_json['master_url'])
    response = requests.get(master_url.replace('"',''))
    #response = requests.get('https://api.discogs.com/masters/178929')
    master_json_str = json.dumps(raw, indent=4)
    master_json = json.loads(master_json_str)
    #print(master_json_str)
    #print(master_json) #finally. Master JSON
    album_title = master_json.get("title", "")
    artist_names = ", ".join(artist["name"] for artist in master_json.get("artists", []))
    mbz_query = f"{artist_names} {album_title}"
    #print(type(master_json['artists']['name']))
    mbz_search = mbz.search_release_groups(mbz_query)
    print(mbz_search)
    #print(mbz_search)
    time.sleep(1)


    

{'release-group-list': [{'id': 'bcf4c04c-0bff-370f-aaa8-fd3057b6f4b1', 'type': 'Album', 'ext:score': '100', 'title': 'Flat as a Pancake', 'first-release-date': '1975', 'primary-type': 'Album', 'artist-credit': [{'name': 'Head East', 'artist': {'id': '1f6fdb90-95a2-419e-9994-b406d562caac', 'name': 'Head East', 'sort-name': 'Head East'}}], 'release-list': [{'id': 'e26ec358-6144-459d-9b92-e37bc1d344f0', 'title': 'Flat as a Pancake', 'status': 'Official'}, {'id': '46afcc21-1dda-42ee-8d70-6ac3c06d2722', 'title': 'Flat as a Pancake', 'status': 'Official'}, {'id': '7ef2ea69-2c7b-4685-b073-1fddb46167c1', 'title': 'Flat as a Pancake', 'status': 'Official'}, {'id': '1b44df31-4f44-4c89-b652-19c506a99fb3', 'title': 'Flat as a Pancake', 'status': 'Official'}, {'id': '9e082db5-6dea-4904-997c-8a939d987ef4', 'title': 'Flat as a Pancake', 'status': 'Official'}], 'release-count': 5, 'tag-list': [{'count': '1', 'name': 'rock'}, {'count': '5', 'name': 'hard rock'}, {'count': '0', 'name': 'classic rock'}, 